In [5]:
from pyspark.sql import SparkSession


In [7]:
spark = SparkSession.builder.getOrCreate()

In [26]:
brochures_path = "exercise-J/brochures.json"
brochuresDF = spark.read.json(brochures_path)

In [27]:
brochures_click_path = "exercise-J/brochure_clicks.json"
brochuresClickDF = spark.read.json(brochures_click_path)

In [28]:
enters_path = "exercise-J/enters.json"
entersDF = spark.read.json(enters_path)

In [29]:
exits_path = "exercise-J/exits.json"
exitsDF = spark.read.json(exits_path)

In [31]:
page_turns_path = "exercise-J/page_turns.json"
pageTurnsDF = spark.read.json(page_turns_path)

In [137]:
brochuresClickDF.select('user_ident').distinct().count()

23

In [33]:
brochuresClickDF.printSchema()

root
 |-- brochure_click_id: long (nullable = true)
 |-- brochure_click_uuid: string (nullable = true)
 |-- brochure_id: long (nullable = true)
 |-- date_time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- location_device_lat: double (nullable = true)
 |-- location_device_lng: double (nullable = true)
 |-- location_intended_lat: double (nullable = true)
 |-- location_intended_lng: double (nullable = true)
 |-- no_profile: boolean (nullable = true)
 |-- page: long (nullable = true)
 |-- page_type: string (nullable = true)
 |-- restricted_ip: boolean (nullable = true)
 |-- traffic_source_type: string (nullable = true)
 |-- traffic_source_value: string (nullable = true)
 |-- treatment: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- user_ident: string (nullable = true)
 |-- user_zip: string (nullable = true)
 |-- visit_id: long (nullable = true)
 |-- visit_origin_type: string (nullable = true)



In [34]:
entersDF.printSchema()

root
 |-- brochure_click_uuid: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- page: long (nullable = true)
 |-- page_view_mode: string (nullable = true)
 |-- screen_size: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [39]:
exitsDF.printSchema()

root
 |-- brochure_click_uuid: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- page: long (nullable = true)
 |-- page_view_mode: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [40]:
pageTurnsDF.printSchema()

root
 |-- brochure_click_uuid: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- page: long (nullable = true)
 |-- page_view_mode: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [41]:
brochuresClickDF.count()

30

In [42]:
entersDF.count()

30

In [43]:
exitsDF.count()

23

In [44]:
pageTurnsDF.count()

317

In [62]:
pageTurnsClick = pageTurnsDF.join(brochuresClickDF, pageTurnsDF.brochure_click_uuid == brochuresClickDF.brochure_click_uuid, "inner")

In [131]:
from pyspark.sql.functions import col, when

In [99]:
pageTurnsClick = brochuresClickDF.alias('broch').join(pageTurnsDF.alias('turn'), col('broch.brochure_click_uuid') == col('turn.brochure_click_uuid'), "right").select([col('broch.user_ident')] + [col('turn.'+col_name) for col_name in pageTurnsDF.columns])

In [100]:
pageTurnsClick.count()

317

In [110]:
pageTurnsClick.select('user_ident').distinct().count()

14

In [210]:
userAcessCountDF = pageTurnsClick.alias('pturn').groupBy(col('pturn.user_ident'), col('pturn.page_view_mode')).count()

In [211]:
userAcessCountDF.count()

14

In [212]:
userAcessCountDF.show()

+--------------------+----------------+-----+
|          user_ident|  page_view_mode|count|
+--------------------+----------------+-----+
|     146360126356971|DOUBLE_PAGE_MODE|   30|
|F85CBA86-4235-419...|SINGLE_PAGE_MODE|    3|
|a4e2a425-20bc-4f9...|SINGLE_PAGE_MODE|    8|
|84361a5c-77a3-426...|DOUBLE_PAGE_MODE|   22|
|c1e2e57d-fc17-472...|SINGLE_PAGE_MODE|   31|
|b27f05d4-0754-42b...|SINGLE_PAGE_MODE|   10|
|139aea9c-fc2a-426...|SINGLE_PAGE_MODE|    1|
|a53bc825-baa2-4f8...|SINGLE_PAGE_MODE|   32|
|22c49150-dcf7-4ed...|SINGLE_PAGE_MODE|   33|
|1d11ba6f-d70d-42b...|SINGLE_PAGE_MODE|   14|
|7C2F34AC-7D0B-40F...|SINGLE_PAGE_MODE|   14|
|98dbada0-46ad-4ec...|SINGLE_PAGE_MODE|   54|
|b513b27e-470f-405...|SINGLE_PAGE_MODE|   19|
|1DD2DCD8-78D1-440...|SINGLE_PAGE_MODE|   46|
+--------------------+----------------+-----+



In [215]:
userAcessAggDF = userAcessCountDF.withColumn('aggregate', when(col('page_view_mode')=='DOUBLE_PAGE_MODE', col('count')*2).when(col('page_view_mode')=='SINGLE_PAGE_MODE', col('count')*1).otherwise(0))

In [216]:
userAcessAggDF.count()

14

In [219]:
userAcessAggDF.groupBy('user_ident').sum('aggregate').show()

+--------------------+--------------+
|          user_ident|sum(aggregate)|
+--------------------+--------------+
|a53bc825-baa2-4f8...|            32|
|F85CBA86-4235-419...|             3|
|b513b27e-470f-405...|            19|
|1d11ba6f-d70d-42b...|            14|
|c1e2e57d-fc17-472...|            31|
|b27f05d4-0754-42b...|            10|
|1DD2DCD8-78D1-440...|            46|
|a4e2a425-20bc-4f9...|             8|
|7C2F34AC-7D0B-40F...|            14|
|84361a5c-77a3-426...|            44|
|22c49150-dcf7-4ed...|            33|
|98dbada0-46ad-4ec...|            54|
|     146360126356971|            60|
|139aea9c-fc2a-426...|             1|
+--------------------+--------------+



In [138]:
entersClick = brochuresClickDF.alias('broch').join(entersDF.alias('enter'), col('broch.brochure_click_uuid') == col('enter.brochure_click_uuid'), "right").select([col('broch.user_ident')] + [col('enter.'+col_name) for col_name in entersDF.columns])

In [142]:
entersClick.select('user_ident').distinct().count()

23

In [143]:
entersClick.show()

+--------------------+--------------------+--------------------+----------+--------------+----+----------------+-----------+--------------------+
|          user_ident| brochure_click_uuid|           date_time|     event|    ip_address|page|  page_view_mode|screen_size|          user_agent|
+--------------------+--------------------+--------------------+----------+--------------+----+----------------+-----------+--------------------+
|1DD2DCD8-78D1-440...|015242a1-ef18-435...|2016-07-12 02:20:...|ENTER_VIEW|  134.3.157.XX|   1|SINGLE_PAGE_MODE|       null|KaufDA/1 CFNetwor...|
|84361a5c-77a3-426...|0017a3da-f18a-4fc...|2016-07-12 18:26:...|ENTER_VIEW|  134.3.254.XX|   1|SINGLE_PAGE_MODE|       null|samsung Android(r...|
|     146360126356971|0092f04a-c2e7-40d...|2016-07-13 01:13:...|ENTER_VIEW| 109.193.87.XX|   0|DOUBLE_PAGE_MODE|  1080x1920|Mozilla/5.0 (Wind...|
|     146360126356971|008421b4-ae52-45c...|2016-07-12 11:32:...|ENTER_VIEW| 109.193.87.XX|   0|DOUBLE_PAGE_MODE|  1080x1920|

In [148]:
entersCountDF = entersClick.alias('penter').groupBy(col('penter.user_ident'), col('penter.page_view_mode')).count()

In [174]:
entersCountDF.printSchema()

root
 |-- user_ident: string (nullable = true)
 |-- page_view_mode: string (nullable = true)
 |-- count: long (nullable = false)



In [203]:
entersAggDF = entersCountDF.withColumn('aggregate', when(col('page_view_mode')=='DOUBLE_PAGE_MODE', col('count')*2).when(col('page_view_mode')=='SINGLE_PAGE_MODE', col('count')*1).otherwise(0))

In [204]:
entersAggDF.select('user_ident').distinct().count()

17

In [205]:
entersAggDF.show()

+--------------------+----------------+-----+---------+
|          user_ident|  page_view_mode|count|aggregate|
+--------------------+----------------+-----+---------+
|37EB8482-47B1-48D...|SINGLE_PAGE_MODE|    1|        1|
|     146360126356971|DOUBLE_PAGE_MODE|    2|        4|
|F85CBA86-4235-419...|SINGLE_PAGE_MODE|    1|        1|
|a4e2a425-20bc-4f9...|SINGLE_PAGE_MODE|    1|        1|
|01475b09-aa3e-432...|SINGLE_PAGE_MODE|    1|        1|
|c1e2e57d-fc17-472...|SINGLE_PAGE_MODE|    1|        1|
|84361a5c-77a3-426...|SINGLE_PAGE_MODE|    2|        2|
|22c49150-dcf7-4ed...|SINGLE_PAGE_MODE|    1|        1|
|     144923378963272|DOUBLE_PAGE_MODE|    1|        2|
|4dc8ec5d-303d-4b4...|SINGLE_PAGE_MODE|    1|        1|
|1d11ba6f-d70d-42b...|SINGLE_PAGE_MODE|    1|        1|
|7C2F34AC-7D0B-40F...|SINGLE_PAGE_MODE|    1|        1|
|98dbada0-46ad-4ec...|SINGLE_PAGE_MODE|    3|        3|
|a1085c7a-3116-4d0...|SINGLE_PAGE_MODE|    3|        3|
|b513b27e-470f-405...|SINGLE_PAGE_MODE|    1|   

In [209]:
entersAggDF.groupBy('user_ident').sum('aggregate').show()

+--------------------+--------------+
|          user_ident|sum(aggregate)|
+--------------------+--------------+
|01475b09-aa3e-432...|             1|
|F85CBA86-4235-419...|             1|
|7857bdd0-c7f0-4a2...|             1|
|b513b27e-470f-405...|             1|
|1d11ba6f-d70d-42b...|             1|
|c1e2e57d-fc17-472...|             1|
|4dc8ec5d-303d-4b4...|             1|
|1DD2DCD8-78D1-440...|             1|
|a4e2a425-20bc-4f9...|             1|
|a1085c7a-3116-4d0...|             3|
|7C2F34AC-7D0B-40F...|             1|
|84361a5c-77a3-426...|             2|
|22c49150-dcf7-4ed...|             1|
|98dbada0-46ad-4ec...|             3|
|     146360126356971|             4|
|     144923378963272|             2|
|37EB8482-47B1-48D...|             1|
+--------------------+--------------+



In [145]:
exitsClick = brochuresClickDF.alias('broch').join(exitsDF.alias('exit'), col('broch.brochure_click_uuid') == col('exit.brochure_click_uuid'), "right").select([col('broch.user_ident')] + [col('exit.'+col_name) for col_name in exitsDF.columns])

In [146]:
exitsClick.select('user_ident').distinct().count()

17

In [147]:
exitsClick.show()

+--------------------+--------------------+--------------------+---------+-------------+----+----------------+--------------------+
|          user_ident| brochure_click_uuid|           date_time|    event|   ip_address|page|  page_view_mode|          user_agent|
+--------------------+--------------------+--------------------+---------+-------------+----+----------------+--------------------+
|     144923378963272|014282ea-1a05-4e5...|2016-07-12 09:12:...|EXIT_VIEW|141.113.11.XX|   0|DOUBLE_PAGE_MODE|Mozilla/5.0 (Wind...|
|22c49150-dcf7-4ed...|00555189-fdb4-434...|2016-07-12 13:38:...|EXIT_VIEW|   46.5.16.XX|  28|SINGLE_PAGE_MODE|kaufda Android(re...|
|F85CBA86-4235-419...|00f5699e-12c7-473...|2016-07-13 00:06:...|EXIT_VIEW|93.235.241.XX|  41|SINGLE_PAGE_MODE|KaufDA/1 CFNetwor...|
|1d11ba6f-d70d-42b...|01163058-2f9c-4ed...|2016-07-12 12:44:...|EXIT_VIEW| 134.3.254.XX|  15|SINGLE_PAGE_MODE|kaufda Android(re...|
|b513b27e-470f-405...|00e0f9ca-d4b9-47f...|2016-07-12 23:57:...|EXIT_VIEW|95

In [186]:
exitsCountDF = exitsClick.alias('pexit').groupBy(col('pexit.user_ident'), col('pexit.page_view_mode')).count()

In [196]:
from pyspark.sql.types import IntegerType

In [197]:
exitsAggDF = exitsCountDF.withColumn('aggregate', when(col('page_view_mode')=='DOUBLE_PAGE_MODE', col('count')*2).when(col('page_view_mode')=='SINGLE_PAGE_MODE', col('count')*1).otherwise(0))

In [198]:
exitsAggDF.select('user_ident').distinct().count()

17

In [199]:
exitsAggDF.show()

+--------------------+----------------+-----+---------+
|          user_ident|  page_view_mode|count|aggregate|
+--------------------+----------------+-----+---------+
|37EB8482-47B1-48D...|SINGLE_PAGE_MODE|    1|        1|
|     146360126356971|DOUBLE_PAGE_MODE|    2|        4|
|F85CBA86-4235-419...|SINGLE_PAGE_MODE|    1|        1|
|a4e2a425-20bc-4f9...|SINGLE_PAGE_MODE|    1|        1|
|01475b09-aa3e-432...|SINGLE_PAGE_MODE|    1|        1|
|c1e2e57d-fc17-472...|SINGLE_PAGE_MODE|    1|        1|
|84361a5c-77a3-426...|SINGLE_PAGE_MODE|    2|        2|
|22c49150-dcf7-4ed...|SINGLE_PAGE_MODE|    1|        1|
|     144923378963272|DOUBLE_PAGE_MODE|    1|        2|
|4dc8ec5d-303d-4b4...|SINGLE_PAGE_MODE|    1|        1|
|1d11ba6f-d70d-42b...|SINGLE_PAGE_MODE|    1|        1|
|7C2F34AC-7D0B-40F...|SINGLE_PAGE_MODE|    1|        1|
|98dbada0-46ad-4ec...|SINGLE_PAGE_MODE|    3|        3|
|a1085c7a-3116-4d0...|SINGLE_PAGE_MODE|    3|        3|
|b513b27e-470f-405...|SINGLE_PAGE_MODE|    1|   

In [200]:
exitsAggDF.printSchema()

root
 |-- user_ident: string (nullable = true)
 |-- page_view_mode: string (nullable = true)
 |-- count: long (nullable = false)
 |-- aggregate: long (nullable = false)



In [202]:
exitsAggDF.groupBy('user_ident').sum('aggregate').show()

+--------------------+--------------+
|          user_ident|sum(aggregate)|
+--------------------+--------------+
|01475b09-aa3e-432...|             1|
|F85CBA86-4235-419...|             1|
|7857bdd0-c7f0-4a2...|             1|
|b513b27e-470f-405...|             1|
|1d11ba6f-d70d-42b...|             1|
|c1e2e57d-fc17-472...|             1|
|4dc8ec5d-303d-4b4...|             1|
|1DD2DCD8-78D1-440...|             1|
|a4e2a425-20bc-4f9...|             1|
|a1085c7a-3116-4d0...|             3|
|7C2F34AC-7D0B-40F...|             1|
|84361a5c-77a3-426...|             2|
|22c49150-dcf7-4ed...|             1|
|98dbada0-46ad-4ec...|             3|
|     146360126356971|             4|
|     144923378963272|             2|
|37EB8482-47B1-48D...|             1|
+--------------------+--------------+

